In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,147194.85,-0.94,-0.10,0.49,-0.01,-0.06,0.01,-0.00,-0.00
3,-0.94,31384.24,-0.17,0.08,-0.07,0.01,0.02,0.00,0.00
4,-0.10,-0.17,6517.69,0.09,-0.00,-0.00,-0.01,-0.00,0.00
5,0.49,0.08,0.09,1311.66,0.00,-0.01,0.01,0.00,0.00
6,-0.01,-0.07,-0.00,0.00,249.28,-0.00,-0.00,0.00,0.00
7,-0.06,0.01,-0.00,-0.01,-0.00,45.17,0.00,0.00,0.00
8,0.01,0.02,-0.01,0.01,-0.00,0.00,7.77,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,1.23,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,-0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00001
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00003,-0.00001,0.00001
5,0.00004,0.00001,0.00003,1.00000,0.00001,-0.00002,0.00005,0.00000,0.00001
6,-0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
7,-0.00002,0.00001,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00001,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00001,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.5446798873426064

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.40923901e+09 1.07755048e+08 4.55263905e+06 1.79275242e+05
 6.21804890e+03 1.91559557e+02 5.05066670e+00 9.57783587e-02
 3.26407468e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999595,-0.027888,-0.005507,-0.001103,-0.000210,-0.000038,-0.000007,-0.000001,-1.285061e-07
1,0.027696,0.999113,-0.031130,-0.005964,-0.001134,-0.000205,-0.000035,-0.000006,-6.642035e-07
2,0.006320,0.030730,0.998871,-0.035080,-0.006367,-0.001152,-0.000199,-0.000032,-3.762153e-06
3,0.001477,0.006951,0.034586,0.998568,-0.039589,-0.006878,-0.001177,-0.000187,-2.256044e-05
4,0.000337,0.001584,0.007623,0.038960,0.998073,-0.047002,-0.007766,-0.001229,-1.484450e-04
5,0.000077,0.000357,0.001721,0.008563,0.046136,0.997099,-0.059218,-0.009032,-1.089389e-03
6,0.000017,0.000081,0.000394,0.001949,0.010285,0.057849,0.994917,-0.081209,-9.443184e-03
7,0.000004,0.000019,0.000093,0.000461,0.002425,0.013395,0.078807,0.989422,-1.210251e-01
8,0.000001,0.000005,0.000023,0.000113,0.000593,0.003271,0.019008,0.119855,9.926039e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00040475321797206476,
 0.000886929475442999,
 0.0011290551705939178,
 0.0014323701234676411,
 0.0019267038242506418,
 0.0029014793266215477,
 0.005083122725092704,
 0.010577532006827872,
 0.007396083150743249]